In [18]:
%matplotlib inline

from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.utils import np_utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

HEIGHT = 224
WIDTH = 224

In [24]:
train = pd.read_csv('train.tsv', sep='\t', header=0)
test = pd.read_csv('test.tsv', sep='\t', header=0)
print(test.size)
test.head()

820


,file
0,7263.jpeg
1,7264.jpeg
2,7265.jpeg
3,7266.png
4,7267.jpeg


In [20]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(HEIGHT, WIDTH))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(prefix, img_paths):
    list_of_tensors = [path_to_tensor(prefix + img_path) for img_path in tqdm_notebook(img_paths)]
    return preprocess_input(np.vstack(list_of_tensors))

X_test = paths_to_tensor("test/", np.array(test['file']))

C:\app\Anaconda3\envs\picnic-hackathon\lib\site-packages\PIL\Image.py:969: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


In [25]:
labels = sorted(train['label'].unique())
labels

['Asparagus, string beans & brussels sprouts',
 'Bananas, apples & pears',
 'Bell peppers, zucchinis & eggplants',
 'Berries & cherries',
 'Broccoli, cauliflowers, carrots & radish',
 'Cheese',
 'Citrus fruits',
 'Cucumber, tomatoes & avocados',
 'Eggs',
 'Fish',
 'Fresh bread',
 'Fresh herbs',
 'Kiwis, grapes & mango',
 'Lunch & Deli Meats',
 'Milk',
 'Minced meat & meatballs',
 'Nectarines, peaches & apricots',
 'Onions, leek, garlic & beets',
 'Pineapples, melons & passion fruit',
 'Pork, beef & lamb',
 'Potatoes',
 'Poultry',
 'Pre-baked breads',
 'Pudding, yogurt & quark',
 'Salad & cress']

In [26]:
one_hot_encoding = preprocessing.LabelBinarizer()
one_hot_encoding.fit(labels)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [27]:
print(len(test))
print(len(X_test))

820
820


In [30]:
model = Sequential()
model.add(ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3)))
for layer in model.layers:
    layer.trainable = False

model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(labels), activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 25)                25625     
Total params: 25,711,513
Trainable params: 2,123,801
Non-trainable params: 23,587,712
_________________________________________________________________


In [31]:
model.load_weights('saved_models/weights.best.Resnet50.hdf5')

In [57]:
def predict(row):
    prediction = model.predict(np.expand_dims(X_test[row.name], axis=0))
    return one_hot_encoding.inverse_transform(prediction)

test['label'] = test.apply(predict, axis=1)
test.head()

,file,label
0,7263.jpeg,"[Bell peppers, zucchinis & eggplants]"
1,7264.jpeg,[Eggs]
2,7265.jpeg,"[Broccoli, cauliflowers, carrots & radish]"
3,7266.png,[Lunch & Deli Meats]
4,7267.jpeg,[Pre-baked breads]


In [60]:
test['label'] = test['label'].apply(lambda l : l[0])

In [62]:
# write to TSV
test.to_csv('21-03-19.tsv', sep='\t', columns=['file', 'label'], index=False)